In [1]:
import numpy as np
import pandas as pd
import re

from collections import Counter

# Open Files as DataFrames

In [2]:
# Setup names to avoid
acct_names = [col_head.replace(u'\ufeff', '').strip() for col_head in pd.read_csv("data/accounts.csv").columns]
cpq_names = [col_head.replace(u'\ufeff', '').strip() for col_head in pd.read_csv("data/cpqs.csv").columns]
opp_names = [col_head.replace(u'\ufeff', '').strip() for col_head in pd.read_csv("data/opportunities.csv").columns]
bldg_names = [col_head.replace(u'\ufeff', '').strip() for col_head in pd.read_csv("data/buildings.csv").columns]

# Read in from csv
accts = pd.read_csv("data/accounts.csv", names=acct_names, skiprows=1)
cpqs = pd.read_csv("data/cpqs.csv", names=cpq_names, skiprows=1)
opps = pd.read_csv("data/opportunities.csv", names=opp_names, skiprows=1)
bldgs = pd.read_csv("data/buildings.csv", names=bldg_names, skiprows=1)

# Add Features of Interest from Other Dataframes to OPPS dataframe

In [3]:
# Define features of interest from other sheets
features_of_interest = {
                        "DandB Total Employees" : int, 
                        "Latitude" : float ,
                        "Longitude" : float,
                        "Net Classification" : str,
                        "Building Type" : str,
                        "Estimated Build Cost" : float,
                        "X36 MRC List" : float, 
                        "X36 NRR List" : float, 
                        "X36 NPV List" : float,
                        "Industry" : str, 
                        "Vertical" : str, 
                        "AnnualRevenue" : float, 
                        "NumberOfEmployees" : float 
                        }

# Append series to DataFrame for each feature of interest
for feature_of_interest, feature_dtype in features_of_interest.items():
    opps[feature_of_interest] = pd.Series(dtype=feature_dtype)

## Define number of rows in dataframe

In [4]:
number_of_observations = len(opps)

number_of_observations

5345

In [5]:
opps.head(3)

,Opportunity ID,Account ID,StageName,IsClosed,IsWon,CreatedDate,Term in Months,Service,Opportunity Type,Product Group,...,X36 NPV List,Net Classification,NumberOfEmployees,Vertical,X36 MRC List,Estimated Build Cost,Latitude,Building Type,Industry,DandB Total Employees
0,Opp-000001,Acct-000007,Closed - Lost,True,False,7/1/16,12.0,NaN,New Service,Ethernet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Opp-000002,Acct-000986,5 - Accepted,True,True,7/1/16,60.0,NaN,New Service,Dark Fiber - Metro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Opp-000002,Acct-000986,5 - Accepted,True,True,7/1/16,60.0,NaN,New Service,Dark Fiber - Metro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Clean and add column data from other sheets for respective features of interest

In [ ]:
for i in range(0, number_of_observations):
    account_id = opps["Account ID"][i]
    building_id = opps["Building ID"][i]
    
    value_1  = list(accts[accts["Account ID"]==account_id]["DandB Total Employees"])
    value_2  = list(bldgs[bldgs["Building ID"]==building_id]["Net Classification"])
    value_3  = list(bldgs[bldgs["Building ID"]==building_id]["Type"])
    value_4  = list(bldgs[bldgs["Building ID"]==building_id]["Estimated Build Cost"])
    value_5  = list(cpqs[cpqs["Account ID"]==account_id]["X36 MRC List"])
    value_6  = list(cpqs[cpqs["Account ID"]==account_id]["X36 NRR List"])
    value_7  = list(cpqs[cpqs["Account ID"]==account_id]["X36 NPV List"])
    value_8  = list(accts[accts["Account ID"]==account_id]["Industry"])
    value_9  = list(accts[accts["Account ID"]==account_id]["Vertical"])
    value_10 = list(accts[accts["Account ID"]==account_id]["Total BRR"])
    value_11 = list(accts[accts["Account ID"]==account_id]["AnnualRevenue"])
    value_12 = list(accts[accts["Account ID"]==account_id]["NumberOfEmployees"])
    value_13 = list(bldgs[bldgs["Building ID"]==building_id]["Latitude"])
    value_14 = list(bldgs[bldgs["Building ID"]==building_id]["Longitude"])
    
    # Check that city value is a string - if not, set to unknown
    if type(opps["City"][i])!=str:
        opps.set_value(i, "City", "unknown")
        
    
    
    """ Process features of interest """
    """ ---------------------------- """
    
    # Process value 1 - DandB Total Employees
    if len(value_1) == 0:
        opps.set_value(i, "DandB Total Employees", 0)
    else:
        opps.set_value(i, "DandB Total Employees", np.average([int(val) for val in value_1]))

    
    # Process value 2 - Net Classification
    if len(value_2) == 0:
        opps.set_value(i, "Net Classification", "unknown")
    else:
        opps.set_value(i, "Net Classification", Counter(value_2).most_common(1)[0][0])
        
        
    # Process value 3 - Building Type
    if len(value_3) == 0:
        opps.set_value(i, "Type", "unknown")
    else:
        opps.set_value(i, "Type", Counter(value_3).most_common(1)[0][0])
        
        
    # Process value 4 - Estimated Build Cost
    if len(value_4) == 0:
        opps.set_value(i, "Estimated Build Cost", 0.0)
    else:
        new_value_4 = [float(re.sub('[!@#$,]', '', val).strip()) for val in value_4]
        opps.set_value(i, "Estimated Build Cost", np.average(new_value_4))
        
        
    # Process value 5 - X36 MRC List
    if len(value_5) == 0:
        opps.set_value(i, "X36 MRC List", 0.0)
    else:
        new_value_5 = [float(re.sub('[!@#$,]', '', val).strip()) for val in value_5]
        opps.set_value(i, "X36 MRC List", np.average(new_value_5))
        
        
    # Process value 6 - X36 NRR List
    if len(value_6) == 0:
        opps.set_value(i, "X36 NRR List", 0.0)
    else:
        new_value_6 = [float(re.sub('[!@#$,]', '', val).strip()) for val in value_6]
        opps.set_value(i, "X36 NRR List", np.average(new_value_6))
        
        
    # Process value 7 - X36 NPV List
    if len(value_7) == 0:
        opps.set_value(i, "X36 NPV List", 0.0)
    else:
        new_value_7 = [float(re.sub('[!@#$,]', '', val).strip()) for val in value_7]
        opps.set_value(i, "X36 NPV List", np.average(new_value_7))
        
        
    # Process value 8 - Industry
    if len(value_8) == 0:
        opps.set_value(i, "Type", "unknown")
    else:
        opps.set_value(i, "Type", Counter(value_8).most_common(1)[0][0])
        
        
    # Process value 9 - Vertical
    if len(value_9) == 0:
        opps.set_value(i, "Vertical", "unknown")
    else:
        opps.set_value(i, "Vertical", Counter(value_9).most_common(1)[0][0])
        
        
    # Process value 10 - Total BRR
    if len(value_10) == 0:
        opps.set_value(i, "Total BRR", 0.0)
    else:
        new_value_10 = [float(re.sub('[!@#$,]', '', val).strip()) for val in value_10]
        opps.set_value(i, "Total BRR", np.average(new_value_10))
        
        
    # Process value 11 - AnnualRevenue
    if len(value_11) == 0:
        opps.set_value(i, "AnnualRevenue", 0.0)
    else:
        new_value_11 = [float(re.sub('[!@#$,]', '', val).strip()) for val in value_11]
        opps.set_value(i, "AnnualRevenue", np.average(new_value_11))
        
        
    # Process value 12 - NumberOfEmployees
    if len(value_12) == 0:
        opps.set_value(i, "NumberOfEmployees", 0)
    else:
        opps.set_value(i, "NumberOfEmployees", np.average([int(val) for val in value_12]))
        
        
    # Process value 13 - Latitude
    if len(value_13) == 0:
        opps.set_value(i, "Latitude", 0.0)
    else:
        opps.set_value(i, "Latitude", np.average([float(val) for val in value_13]))
        
        
    # Process value 14 - Longitude
    if len(value_14) == 0:
        opps.set_value(i, "Longitude", 0.0)
    else:
        opps.set_value(i, "Longitude", np.average([float(val) for val in value_14]))
    
    
    
    
    """ Process features in opps df """
    """ --------------------------- """
    
    # Process network proximity
    if np.isnan(float(opps["Network Proximity"][i])):
        opps.set_value(i, "Network Proximity", -1)

        
    # Process opportunity type
    if type(opps["Opportunity Type"][i])==float:
        opps.set_value(i, "Opportunity Type", "unknown")

        
    # Process Term 
    if np.isnan(float(opps["Term in Months"][i])):
        opps.set_value(i, "Term in Months", -1)
    else:
        opps.set_value(i, "Term in Months", float(opps["Term in Months"][i]))

### Make backup

In [ ]:
opps_backup = opps.copy(deep=True)

In [ ]:
opps_backup

# Save to CSV

In [ ]:
opps.to_csv("data/concat_data.csv")